Transformed from linux machine for simpler setup:
Contains a simple implementation of cipher text processing residual neural network.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random

Frequency: how many 1s in the sequence
runs: length of the longest sub-sequence of 1 in the sequence
serial: for n in range(length): test how many matches in 


In [ ]:
#activation function types
#In original paper, they used relu activation
#Different activation function is used for parameter tuning
def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

In [ ]:
from torch.serialization import INT_SIZE
'''
From Xia et.al's implementation, the residual network shows as below
convolutional 1d layer
batch normalization layer
activation layer
'''
class ResBlock(nn.Module):

    def __init__(self, size,activation='relu',downsampling=1):
        super().__init__()
        #paper structure
        self.conv1 = nn.Conv1d(size, size, 1, padding=0,stride=downsampling)
        self.batchnorm1 = nn.BatchNorm1d(size)
        self.activate = activation_func(activation)
        #apply identity activation to uniform the sizes
        self.blocks=nn.Identity()
    def activation(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        return x
   
    """
    Combine output with the original input
    """
    def forward(self, x): 
      return x+ self.activation(x) 

In [ ]:
class ResNetLayer(nn.Module):
    """
    A ResNet layer composed by `n` blocks stacked one after the other
    """
    def __init__(self, channels, block=ResBlock, n=1, *args, **kwargs):
        super().__init__()
        self.blocks = nn.Sequential(
            block(channels ,*args, **kwargs),
            *[block(channels, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [ ]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by layers with increasing features.
    """
    def __init__(self, blocks_size,
                 activation='relu', block=ResBlock,n=1, *args, **kwargs):
        super().__init__()
        self.blocks_size = blocks_size
        
        self.gate = nn.Sequential(
            nn.Conv1d(blocks_size, blocks_size,1,padding=0),
            nn.BatchNorm1d(self.blocks_size),
            activation_func(activation)
        )
        
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_size, n=n, activation=activation, 
                        *args, **kwargs),
            *[ResNetLayer(blocks_size, 
                         n=n, activation=activation, 
                          *args, **kwargs) for _ in range(n - 1)]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

In [ ]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

In [ ]:
m = ResNetEncoder(3,n=5)
input = torch.randn(3,3,4)
output = m(input)
output

tensor([[[ 3.1906,  4.6163,  5.3328,  3.3847],
         [ 7.4579,  7.0185,  6.3493,  7.3638],
         [ 5.4903,  6.8321,  5.2643,  5.3586]],

        [[ 3.4586, 37.5507,  3.1800,  3.3623],
         [ 8.0851, 17.0998,  7.4518,  7.3301],
         [ 6.2925, 24.4898,  5.4314,  5.3373]],

        [[ 3.1478, 34.1901,  8.2235,  3.3187],
         [ 7.5069, 22.5154,  5.9799,  7.2647],
         [ 5.4272, 32.6679,  6.2005,  5.2969]]], grad_fn=<AddBackward0>)

In [ ]:
'''
Feature Engineering
'''
